In [1]:
import numpy as np
import pandas as pd
import datetime
import time
from tqdm import tqdm
%run ./../custom_functions/train_dev_test_split.ipynb
%run ./../custom_functions/get_future_predictions.ipynb

from pmdarima import auto_arima

In [2]:
marag_df = pd.read_csv('./../../../Databases/clean_data/marag_lags_data.csv')
marag_df.set_index('Timestamp', inplace=True)
marag_df.sort_index(inplace=True)

marag_daily = pd.read_csv('./../../../Databases/clean_data/marag_dailylags_data.csv')
marag_daily.set_index('Timestamp', inplace=True)
marag_daily.sort_index(inplace=True)

# For the time being we will not use the outliers column, so we will drop it
marag_df.drop(['outliers'], axis=1, inplace=True)

In [3]:
# Let's import the marag data with no lags for future comparative purposes
no_lags_marag = pd.read_csv('./../../../Databases/clean_data/marag_data.csv')
no_lags_marag.set_index('Timestamp', inplace=True)
no_lags_marag.sort_index(inplace=True)
no_lags_marag.drop(['outliers'], axis=1, inplace=True)

daily_nolags_marag = pd.read_csv('./../../../Databases/clean_data/marag_daily_data.csv')
daily_nolags_marag.set_index('Timestamp', inplace=True)
daily_nolags_marag.sort_index(inplace=True)

In [4]:
marag_daily.head()

,TotalEntries,T,PPT,Weekday/Weekend,t-1,t-2,t-3,t-4,t-5,t-6,t-7
Timestamp,,,,,,,,,,,
2017-01-16,7785,9.856250,0.000000,1,3248.0,3980.0,5585.0,1049.0,1144.0,1117.0,182.0
2017-01-17,7710,6.158333,0.000000,1,7785.0,3248.0,3980.0,5585.0,1049.0,1144.0,1117.0
2017-01-18,7785,6.150000,0.000000,1,7710.0,7785.0,3248.0,3980.0,5585.0,1049.0,1144.0
2017-01-19,7733,8.356250,0.000000,1,7785.0,7710.0,7785.0,3248.0,3980.0,5585.0,1049.0
2017-01-20,8161,9.722917,0.064583,1,7733.0,7785.0,7710.0,7785.0,3248.0,3980.0,5585.0


In [5]:
# Let's create a dataframe in which we'll append all the estimated predictions, and another
# containing the running times of each scenario.

ARIMA_results_df = pd.DataFrame
ARIMA_runtime_df = pd.DataFrame
ARIMA_confint_df = pd.DataFrame

ARIMA_dailyresults_df = pd.DataFrame
ARIMA_runtime_dailydf = pd.DataFrame
ARIMA_confint_dailydf = pd.DataFrame

ARIMA_noexogenous_df = pd.DataFrame
ARIMA_runtime_noexo = pd.DataFrame
ARIMA_noexo_confint = pd.DataFrame

## Model tuning

In [11]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])
ARIMA_X_train = pd.concat([X_train, X_dev])
ARIMA_X_train.sort_index(inplace=True)
ARIMA_y_train = pd.concat([y_train, y_dev])
ARIMA_y_train.sort_index(inplace=True)

In [12]:
# Model with exogenous data (allcols)

start_ARIMA_tune = time.time()
ARIMA_model = auto_arima(ARIMA_y_train, exogenous=ARIMA_X_train, start_p=1, start_q=1,
                           start_P=0, seasonal=True, suppress_warnings=True,
                           error_action='ignore', trace=True, stepwise=True)

end_ARIMA_tune = time.time()
ARIMA_tune_runtime = end_ARIMA_tune - start_ARIMA_tune

Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=285362.755, BIC=288199.456, Fit time=141.187 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=285121.586, BIC=287966.581, Fit time=742.116 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=285234.934, BIC=288079.930, Fit time=815.175 seconds
Fit ARIMA: order=(2, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=284832.314, BIC=287685.604, Fit time=813.882 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=284334.622, BIC=287204.501, Fit time=894.791 seconds
Fit ARIMA: order=(4, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=284064.070, BIC=286942.244, Fit time=862.785 seconds
Fit ARIMA: order=(4, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=284003.310, BIC=286873.189, Fit time=834.179 seconds
Fit ARIMA: order=(5

In [13]:
ARIMA_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                29568
Model:               SARIMAX(3, 0, 0)   Log Likelihood             -141628.865
Date:                Sun, 05 May 2019   AIC                         283947.730
Time:                        21:54:54   BIC                         286809.314
Sample:                             0   HQIC                        284866.537
                              - 29568                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.1520      1.931     -0.079      0.937      -3.936       3.632
x1             0.1669      0.080      2.097      0.036       0.011       0.323
x2            -0.2372      0.362     -0.656      0.512      -0.946       0.472
x3             1.8124      2.619      0.692      0.489      -3.320       6.945
x4            15.1349      1.456     10.393      0.000      12.281      17.989
x5             0.5346      0.025     21.072      0.000       0.485       0.584
x6             0.1326      0.030      4.352      0.000       0.073       0.192
x7            -0.2139      0.033     -6.390      0.000      -0.279      -0.148
x8             0.0665      0.019      3.559      0.000       0.030       0.103
x9             0.0172      0.013      1.341      0.180      -0.008       0.042
x10            0.0311      0.013      2.314      0.021       0.005       0.057
x11           -0.0378      0.008     -4.715      0.000      -0.053      -0.022
x12           -0.0121      0.007     -1.684      0.092      -0.026       0.002
x13           -0.0123      0.007     -1.677      0.094      -0.027       0.002
x14            0.0211      0.008      2.748      0.006       0.006       0.036
x15            0.0017      0.008      0.225      0.822      -0.013       0.017
x16            0.0380      0.008      4.871      0.000       0.023       0.053
x17           -0.0265      0.008     -3.429      0.001      -0.042      -0.011
x18            0.0468      0.008      6.141      0.000       0.032       0.062
x19            0.0133      0.008      1.697      0.090      -0.002       0.029
x20           -0.0145      0.008     -1.862      0.063      -0.030       0.001
x21            0.0079      0.008      1.025      0.305      -0.007       0.023
x22           -0.0507      0.007     -6.940      0.000      -0.065      -0.036
x23           -0.0005      0.008     -0.061      0.951      -0.016       0.015
x24            0.0515      0.008      6.817      0.000       0.037       0.066
x25            0.0066      0.008      0.813      0.416      -0.009       0.023
x26           -0.0026      0.008     -0.328      0.743      -0.018       0.013
x27            0.0018      0.008      0.213      0.831      -0.014       0.018
x28           -0.0009      0.008     -0.105      0.917      -0.017       0.015
x29            0.0191      0.008      2.313      0.021       0.003       0.035
x30            0.0060      0.008      0.719      0.472      -0.010       0.022
x31           -0.0133      0.008     -1.580      0.114      -0.030       0.003
x32            0.0175      0.008      2.118      0.034       0.001       0.034
x33           -0.0241      0.008     -2.859      0.004      -0.041      -0.008
x34            0.0047      0.009      0.541      0.589      -0.012       0.022
x35           -0.0125      0.008     -1.512      0.130      -0.029       0.004
x36           -0.0176      0.008     -2.175      0.030      -0.033      -0.002
x37            0.0207      0.008      2.653      0.008       0.005       0.036
x3

In [14]:
# Model without exogenous data

start_noexo_ARIMA_tune = time.time()
noexo_ARIMA_model = auto_arima(ARIMA_y_train, start_p=1, start_q=1,
                           start_P=0, seasonal=True, suppress_warnings=True,
                           error_action='ignore', trace=True, stepwise=True)

end_noexo_ARIMA_tune = time.time()
ARIMA_noexo_tune_runtime = end_noexo_ARIMA_tune - start_noexo_ARIMA_tune

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=314829.435, BIC=314862.613, Fit time=7.916 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=315812.000, BIC=315828.589, Fit time=0.499 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=315658.462, BIC=315683.346, Fit time=0.598 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=315717.475, BIC=315742.358, Fit time=1.416 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=312369.001, BIC=312410.473, Fit time=7.723 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=313031.296, BIC=313064.473, Fit time=1.226 seconds
Fit ARIMA: order=(2, 1, 2) seasonal_order=(0, 0, 0, 1); AIC=312153.196, BIC=312202.963, Fit time=11.018 seconds
Fit ARIMA: order=(3, 1, 3) seasonal_order=(0, 0, 0, 1); AIC=311401.820, BIC=311468.175, Fit time=17.737 seconds
Fit ARIMA: order=(2, 1, 3) seasonal_order=(0, 0, 0, 1); AIC=311457.373, BIC=311515.434, Fit time=27.559 second

In [15]:
noexo_ARIMA_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                29568
Model:               SARIMAX(4, 1, 3)   Log Likelihood             -155553.855
Date:                Sun, 05 May 2019   AIC                         311125.710
Time:                        21:57:31   BIC                         311200.360
Sample:                             0   HQIC                        311149.679
                              - 29568                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -0.0091      0.726     -0.013      0.990      -1.431       1.413
ar.L1         -0.6778      0.020    -33.556      0.000      -0.717      -0.638
ar.L2          0.2036      0.029      7.100      0.000       0.147       0.260
ar.L3         -0.2775      0.013    -21.596      0.000      -0.303      -0.252
ar.L4         -0.1591      0.009    -17.641      0.000      -0.177      -0.141
ma.L1          0.7779      0.020     38.829      0.000       0.739       0.817
ma.L2          0.1918      0.030      6.307      0.000       0.132       0.251
ma.L3          0.4107      0.016     25.647      0.000       0.379       0.442
sigma2      2194.3177      7.829    280.298      0.000    2178.974    2209.661
===================================================================================
Ljung-Box (Q):                     3508.64   Jarque-Bera (JB):            269043.34
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.10   Skew:                             0.42
Prob(H) (two-sided):                  0.00   Kurtosis:                        17.75
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Original data (48 obsv. / day)

### All columns, known future

In [ ]:
ARIMA_model.fit(ARIMA_y_train, exogenous=ARIMA_X_train)

In [17]:
start_ARIMA_PKF_allcols = time.time()
ARIMA_y_hat, ARIMA_conf_int_y = ARIMA_model.predict(n_periods=len(y_test), exogenous=X_test, return_conf_int=True)
ARIMA_y_hat = pd.Series(ARIMA_y_hat, index=y_test.index, name='TotEntr_PKF_allcols')
ARIMA_conf_int_y = pd.DataFrame(ARIMA_conf_int_y, columns=['PKF_allcols_min','PKF_allcols_max'], index=y_test.index)

end_ARIMA_PKF_allcols = time.time()
ARIMA_runtime_PKF_allcols = end_ARIMA_PKF_allcols - start_ARIMA_PKF_allcols
ARIMA_runtime_PKF_allcols = pd.DataFrame(pd.Series(ARIMA_runtime_PKF_allcols, name = 'PKF_allcols_ARIMA_runtime'))

In [18]:
ARIMA_observed_values = pd.DataFrame(y_test)
ARIMA_observed_values.columns = ['Observed_values']
ARIMA_results_df = ARIMA_observed_values.merge(pd.DataFrame(ARIMA_y_hat), how='left', on='Timestamp')
ARIMA_confint_df = ARIMA_conf_int_y
ARIMA_results_df.head()

,Observed_values,TotEntr_PKF_allcols
Timestamp,,
2018-09-24 00:00:00,20,4.183628
2018-09-24 00:30:00,0,-3.999364
2018-09-24 01:00:00,0,-13.922692
2018-09-24 01:30:00,0,-25.213029
2018-09-24 02:00:00,0,-6.003513


In [47]:
ARIMA_runtime_df = ARIMA_runtime_PKF_allcols
ARIMA_runtime_df.head()

,PKF_allcols_ARIMA_runtime
0,810.089135


### Just lags, known future

In [20]:
lags_PKF_df = marag_df.drop(['Temperature', 'Precipitation', 'Open/Closed',
       'Weekday/Weekend'], axis = 1)

X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(lags_PKF_df.drop(['TotalEntries'], axis=1), lags_PKF_df['TotalEntries'])

ARIMA_lags_X_train = pd.concat([X_train, X_dev])
ARIMA_lags_X_train.sort_index(inplace=True)
ARIMA_lags_y_train = pd.concat([y_train, y_dev])
ARIMA_lags_y_train.sort_index(inplace=True)

In [ ]:
ARIMA_lags_model = ARIMA_model.fit(ARIMA_lags_y_train, exogenous=ARIMA_lags_X_train)

In [21]:
start_ARIMA_PKF_lags = time.time()
ARIMA_lags_y_hat, ARIMA_lags_conf_int_y = ARIMA_lags_model.predict(n_periods=len(y_test), exogenous=X_test, return_conf_int=True)
ARIMA_lags_y_hat = pd.Series(ARIMA_lags_y_hat, index=y_test.index, name='TotEntr_PKF_lags')
ARIMA_lags_conf_int_y = pd.DataFrame(ARIMA_lags_conf_int_y, columns=['PKF_lags_min','PKF_lags_max'], index=y_test.index)

end_ARIMA_PKF_lags = time.time()
ARIMA_runtime_PKF_lags = end_ARIMA_PKF_lags - start_ARIMA_PKF_lags
ARIMA_runtime_PKF_lags = pd.DataFrame(pd.Series(ARIMA_runtime_PKF_lags, name = 'PKF_lags_ARIMA_runtime'))

In [22]:
ARIMA_confint_df = ARIMA_confint_df.merge(ARIMA_lags_conf_int_y, how='left', on='Timestamp')
ARIMA_results_df = ARIMA_results_df.merge(pd.DataFrame(ARIMA_lags_y_hat), how='left', on='Timestamp')
ARIMA_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags
Timestamp,,,
2018-09-24 00:00:00,20,4.183628,-1.160668
2018-09-24 00:30:00,0,-3.999364,-7.325107
2018-09-24 01:00:00,0,-13.922692,-14.139623
2018-09-24 01:30:00,0,-25.213029,-25.625291
2018-09-24 02:00:00,0,-6.003513,-5.980766


In [48]:
ARIMA_runtime_df = pd.concat([ARIMA_runtime_df, ARIMA_runtime_PKF_lags], axis=1)
ARIMA_runtime_df.head()

,PKF_allcols_ARIMA_runtime,PKF_lags_ARIMA_runtime
0,810.089135,795.753101


It is not possible to extract results for the predicting unknown future ($\textbf{PUF}$) scenario using all columns. The reason is that around a few more than 900 iterations, an overflow error occurs. Therefore, this section will be omitted.

### Just lags, unknown future

In [27]:
lags_PUF_df = marag_df.drop(['Temperature', 'Precipitation', 'Open/Closed',
       'Weekday/Weekend'], axis = 1)

X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(lags_PKF_df.drop(['TotalEntries'], axis=1), lags_PKF_df['TotalEntries'])

ARIMA_Ulags_X_train = pd.concat([X_train, X_dev])
ARIMA_Ulags_X_train.sort_index(inplace=True)
ARIMA_Ulags_y_train = pd.concat([y_train, y_dev])
ARIMA_Ulags_y_train.sort_index(inplace=True)

In [32]:
ARIMA_Ulags_model = ARIMA_model.fit(ARIMA_Ulags_y_train, exogenous=ARIMA_Ulags_X_train)

In [34]:
start_PUF_lags = time.time()

ARIMA_allfuture_lags_df, ARIMA_confint_lags_future = ARIMA_future_preds(no_lags_marag, ARIMA_Ulags_X_train, X_test, ARIMA_Ulags_model, exogenous=True, justlags=True)

end_PUF_lags = time.time()
ARIMA_runtime_PUF_lags = end_PUF_lags - start_PUF_lags
ARIMA_runtime_PUF_lags = pd.DataFrame(pd.Series(ARIMA_runtime_PUF_lags, name='PUF_lags_ARIMA_runtime'))
ARIMA_allfuture_lags_df.columns = ['TotEntr_PUF_lags']
ARIMA_confint_lags_future.columns = ['PUF_lags_min','PUF_lags_max']


  4%|▍         | 293/7392 [07:35<2:54:07,  1.47s/it]


  8%|▊         | 585/7392 [15:09<2:37:41,  1.39s/it]


 12%|█▏        | 877/7392 [22:33<2:47:50,  1.55s/it]


 16%|█▌        | 1165/7392 [30:13<3:24:46,  1.97s/it]


 20%|█▉        | 1451/7392 [39:38<2:28:58,  1.50s/it]


...

 88%|████████▊ | 6478/7392 [3:13:01<27:40,  1.82s/it]


 92%|█████████▏| 6764/7392 [3:21:26<18:36,  1.78s/it]


 95%|█████████▌| 7050/7392 [3:30:04<09:55,  1.74s/it]


 99%|█████████▉| 7336/7392 [3:38:45<01:44,  1.86s/it]


100%|██████████| 7392/7392 [3:40:30<00:00,  1.86s/it]

In [35]:
ARIMA_confint_lags_future.columns = ['PUF_lags_min','PUF_lags_max']

In [36]:
ARIMA_confint_df = ARIMA_confint_df.merge(ARIMA_confint_lags_future, how='left', on='Timestamp')
ARIMA_results_df = ARIMA_results_df.merge(ARIMA_allfuture_lags_df, how='left', on='Timestamp')
ARIMA_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags,TotEntr_PUF_lags
Timestamp,,,,
2018-09-24 00:00:00,20,4.183628,-1.160668,-1.160668
2018-09-24 00:30:00,0,-3.999364,-7.325107,-18.933325
2018-09-24 01:00:00,0,-13.922692,-14.139623,-27.434277
2018-09-24 01:30:00,0,-25.213029,-25.625291,-38.792805
2018-09-24 02:00:00,0,-6.003513,-5.980766,-28.479983


In [49]:
ARIMA_runtime_df = pd.concat([ARIMA_runtime_df, ARIMA_runtime_PUF_lags], axis=1)
ARIMA_runtime_df.head()

,PKF_allcols_ARIMA_runtime,PKF_lags_ARIMA_runtime,PUF_lags_ARIMA_runtime
0,810.089135,795.753101,13231.050178


### Not using exogenous data

In [39]:
X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(marag_df.drop(['TotalEntries'], axis=1), 
                                                                      marag_df['TotalEntries'])
ARIMA_X_train = pd.concat([X_train, X_dev])
ARIMA_X_train.sort_index(inplace=True)
ARIMA_y_train = pd.concat([y_train, y_dev])
ARIMA_y_train.sort_index(inplace=True)

In [40]:
start_noexo = time.time()

ARIMA_noexo_df, ARIMA_noexo_confint = ARIMA_future_preds(no_lags_marag, ARIMA_X_train, X_test, noexo_ARIMA_model)

end_noexo = time.time()
ARIMA_runtime_noexo = end_noexo - start_noexo
ARIMA_runtime_noexo = pd.DataFrame(pd.Series(ARIMA_runtime_noexo, name='noexo_ARIMA_runtime'))
ARIMA_noexo_df.columns = ['TotEntr_noexo']
ARIMA_noexo_confint.columns = ['Noexo_min','Noexo_max']

In [41]:
ARIMA_confint_df = ARIMA_confint_df.merge(ARIMA_noexo_confint, how='left', on='Timestamp')
ARIMA_results_df = ARIMA_results_df.merge(ARIMA_noexo_df, how='left', on='Timestamp')
ARIMA_results_df.head()

,Observed_values,TotEntr_PKF_allcols,TotEntr_PKF_lags,TotEntr_PUF_lags,TotEntr_noexo
Timestamp,,,,,
2018-09-24 00:00:00,20,4.183628,-1.160668,-1.160668,40.376416
2018-09-24 00:30:00,0,-3.999364,-7.325107,-18.933325,23.993877
2018-09-24 01:00:00,0,-13.922692,-14.139623,-27.434277,35.631315
2018-09-24 01:30:00,0,-25.213029,-25.625291,-38.792805,27.798463
2018-09-24 02:00:00,0,-6.003513,-5.980766,-28.479983,39.318221


In [50]:
ARIMA_runtime_df = pd.concat([ARIMA_runtime_df, ARIMA_runtime_noexo], axis=1)
ARIMA_runtime_df.head()

,PKF_allcols_ARIMA_runtime,PKF_lags_ARIMA_runtime,PUF_lags_ARIMA_runtime,noexo_ARIMA_runtime
0,810.089135,795.753101,13231.050178,0.544242


In [51]:
ARIMA_results_df.to_csv('./../../../Databases/results_data/ARIMA_results.csv')
ARIMA_confint_df.to_csv('./../../../Databases/results_data/ARIMA_confint.csv')
ARIMA_runtime_df.to_csv('./../../../Databases/results_data/ARIMA_runtime_scenarios.csv', index=False)

## Daily data (1 obs. / day)

### Model tuning

In [25]:
X_train, X_dev, dARIMA_X_test, y_train, y_dev, dARIMA_y_test = train_dev_test_split(marag_daily.drop(['TotalEntries'], axis=1), 
                                                                      marag_daily['TotalEntries'])
dARIMA_X_train = pd.concat([X_train, X_dev])
dARIMA_X_train.sort_index(inplace=True)
dARIMA_y_train = pd.concat([y_train, y_dev])
dARIMA_y_train.sort_index(inplace=True)

In [26]:
# Model with exogenous daily data (allcols)

start_dARIMA_tune = time.time()
daily_ARIMA_model = auto_arima(dARIMA_y_train, exogenous=dARIMA_X_train, start_p=1, start_q=1,
                           start_P=0, seasonal=True, suppress_warnings=True,
                           error_action='ignore', trace=True, stepwise=True)

end_dARIMA_tune = time.time()
dARIMA_tune_runtime = end_dARIMA_tune - start_dARIMA_tune

Fit ARIMA: order=(1, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=10332.376, BIC=10394.301, Fit time=0.664 seconds
Fit ARIMA: order=(0, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=10349.402, BIC=10402.481, Fit time=0.144 seconds
Fit ARIMA: order=(1, 0, 0) seasonal_order=(0, 0, 0, 1); AIC=10339.843, BIC=10397.346, Fit time=0.290 seconds
Fit ARIMA: order=(0, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=10342.106, BIC=10399.608, Fit time=0.292 seconds
Fit ARIMA: order=(2, 0, 1) seasonal_order=(0, 0, 0, 1); AIC=10333.294, BIC=10399.642, Fit time=1.024 seconds
Fit ARIMA: order=(1, 0, 2) seasonal_order=(0, 0, 0, 1); AIC=10338.093, BIC=10404.442, Fit time=0.472 seconds
Fit ARIMA: order=(2, 0, 2) seasonal_order=(0, 0, 0, 1); AIC=10335.703, BIC=10406.475, Fit time=1.077 seconds
Total fit time: 3.967 seconds


In [27]:
daily_ARIMA_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  616
Model:               SARIMAX(1, 0, 1)   Log Likelihood               -5152.188
Date:                Thu, 09 May 2019   AIC                          10332.376
Time:                        17:04:51   BIC                          10394.301
Sample:                             0   HQIC                         10356.454
                                - 616                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -25.7786     64.830     -0.398      0.691    -152.842     101.285
x1            34.5051     18.156      1.900      0.057      -1.080      70.090
x2          -888.2108    180.923     -4.909      0.000   -1242.813    -533.608
x3          3180.3515    196.977     16.146      0.000    2794.283    3566.420
x4             0.2539      0.039      6.501      0.000       0.177       0.330
x5             0.0718      0.034      2.117      0.034       0.005       0.138
x6             0.0965      0.036      2.683      0.007       0.026       0.167
x7             0.0671      0.034      1.963      0.050    8.99e-05       0.134
x8             0.0097      0.034      0.282      0.778      -0.058       0.077
x9             0.0036      0.033      0.110      0.912      -0.061       0.068
x10            0.1550      0.036      4.295      0.000       0.084       0.226
ar.L1          0.9043      0.081     11.204      0.000       0.746       1.062
ma.L1         -0.8346      0.102     -8.145      0.000      -1.035      -0.634
sigma2      1.086e+06   4.32e+04     25.159      0.000       1e+06    1.17e+06
===================================================================================
Ljung-Box (Q):                       51.31   Jarque-Bera (JB):              1258.29
Prob(Q):                              0.11   Prob(JB):                         0.00
Heteroskedasticity (H):               0.95   Skew:                            -1.60
Prob(H) (two-sided):                  0.69   Kurtosis:                         9.23
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [12]:
# Model without exogenous daily data

start_noexo_dARIMA_tune = time.time()
noexo_dARIMA_model = auto_arima(dARIMA_y_train, start_p=1, start_q=1,
                           start_P=0, seasonal=True, suppress_warnings=True,
                           error_action='ignore', trace=True, stepwise=True)

end_noexo_dARIMA_tune = time.time()
dARIMA_noexo_tune_runtime = end_noexo_dARIMA_tune - start_noexo_dARIMA_tune

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=11053.731, BIC=11071.417, Fit time=0.331 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=11271.271, BIC=11280.115, Fit time=0.013 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=11261.515, BIC=11274.780, Fit time=0.028 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=11111.612, BIC=11124.877, Fit time=0.147 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=10988.849, BIC=11010.957, Fit time=0.396 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(0, 0, 0, 1); AIC=11160.888, BIC=11178.574, Fit time=0.057 seconds
Fit ARIMA: order=(2, 1, 2) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 1, 2) seasonal_order=(0, 0, 0, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 0, 0, 1); AIC=10989.809, BIC=11016.339, Fit time=0.513 seconds
Total fit time: 1.492 seconds


In [13]:
noexo_dARIMA_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Statespace Model Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  616
Model:               SARIMAX(2, 1, 1)   Log Likelihood               -5489.424
Date:                Thu, 09 May 2019   AIC                          10988.849
Time:                        17:02:28   BIC                          11010.957
Sample:                             0   HQIC                         10997.446
                                - 616                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      2.8862      7.630      0.378      0.705     -12.069      17.842
ar.L1          0.3821      0.044      8.758      0.000       0.297       0.468
ar.L2         -0.3334      0.047     -7.107      0.000      -0.425      -0.241
ma.L1         -0.9106      0.020    -45.815      0.000      -0.950      -0.872
sigma2      3.301e+06   2.75e+05     12.001      0.000    2.76e+06    3.84e+06
===================================================================================
Ljung-Box (Q):                     1432.86   Jarque-Bera (JB):                31.79
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.97   Skew:                            -0.43
Prob(H) (two-sided):                  0.83   Kurtosis:                         2.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

### Daily data: all columns, known future

In [59]:
start_ARIMA_dPKF_allcols = time.time()
dARIMA_y_hat, dARIMA_y_confint = daily_ARIMA_model.predict(n_periods=len(dARIMA_y_test), exogenous=dARIMA_X_test, return_conf_int=True)
dARIMA_y_hat = pd.Series(dARIMA_y_hat, index=dARIMA_y_test.index, name='DayEntr_PKF_allcols')
dARIMA_y_confint = pd.DataFrame(dARIMA_y_confint, columns=['daily_PKF_allcols_min','daily_PKF_allcols_max'], index=dARIMA_y_test.index)

end_ARIMA_dPKF_allcols = time.time()
dARIMA_runtime_PKF_allcols = end_ARIMA_dPKF_allcols - start_ARIMA_dPKF_allcols
dARIMA_runtime_PKF_allcols = pd.DataFrame(pd.Series(dARIMA_runtime_PKF_allcols, name = 'daily_PKF_allcols_ARIMA_runtime'))

In [60]:
dARIMA_observed_values = pd.DataFrame(dARIMA_y_test)
dARIMA_observed_values.columns = ['Observed_values']
ARIMA_dailyresults_df = dARIMA_observed_values.merge(pd.DataFrame(dARIMA_y_hat), how='left', on='Timestamp')
ARIMA_confint_dailydf = dARIMA_y_confint
ARIMA_dailyresults_df.head()

,Observed_values,DayEntr_PKF_allcols
Timestamp,,
2018-09-24,5719,9265.802845
2018-09-25,9998,8657.545503
2018-09-26,9816,9453.451855
2018-09-27,10087,9591.111601
2018-09-28,10742,10171.307107


In [61]:
ARIMA_runtime_dailydf = dARIMA_runtime_PKF_allcols
ARIMA_runtime_dailydf

,daily_PKF_allcols_ARIMA_runtime
0,0.013519


### Daily data: just lags, known future

In [62]:
lags_dPKF_df = marag_daily.drop(['T', 'PPT', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, ARIMA_dlags_X_test, y_train, y_dev, ARIMA_dlags_y_test = train_dev_test_split(lags_dPKF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_dPKF_df['TotalEntries'])

ARIMA_dlags_X_train = pd.concat([X_train, X_dev])
ARIMA_dlags_X_train.sort_index(inplace=True)
ARIMA_dlags_y_train = pd.concat([y_train, y_dev])
ARIMA_dlags_y_train.sort_index(inplace=True)

In [63]:
dARIMA_lags_model = daily_ARIMA_model.fit(ARIMA_dlags_y_train, exogenous = ARIMA_dlags_X_train)

In [64]:
start_dARIMA_PKF_lags = time.time()
dARIMA_lags_y_hat, dARIMA_lags_confint_y = dARIMA_lags_model.predict(n_periods=len(ARIMA_dlags_y_test), exogenous=ARIMA_dlags_X_test, return_conf_int=True)
dARIMA_lags_y_hat = pd.Series(dARIMA_lags_y_hat, index=ARIMA_dlags_y_test.index, name='DayEntr_PKF_lags')
dARIMA_lags_confint_y = pd.DataFrame(dARIMA_lags_confint_y, columns=['daily_PKF_lags_min','daily_PKF_lags_max'], index=ARIMA_dlags_y_test.index)

end_dARIMA_PKF_lags = time.time()
dARIMA_runtime_PKF_lags = end_dARIMA_PKF_lags - start_dARIMA_PKF_lags
dARIMA_runtime_PKF_lags = pd.DataFrame(pd.Series(dARIMA_runtime_PKF_lags, name = 'daily_PKF_lags_ARIMA_runtime'))

In [66]:
ARIMA_dailyresults_df = ARIMA_dailyresults_df.merge(pd.DataFrame(dARIMA_lags_y_hat), how='left', on='Timestamp')
ARIMA_confint_dailydf = ARIMA_confint_dailydf.merge(dARIMA_lags_confint_y, how='left', on='Timestamp')
ARIMA_dailyresults_df.head()

,Observed_values,DayEntr_PKF_allcols,DayEntr_PKF_lags
Timestamp,,,
2018-09-24,5719,9265.802845,9516.854425
2018-09-25,9998,8657.545503,9135.138861
2018-09-26,9816,9453.451855,10047.471310
2018-09-27,10087,9591.111601,10130.052655
2018-09-28,10742,10171.307107,10093.401038


In [67]:
ARIMA_runtime_dailydf = pd.concat([ARIMA_runtime_dailydf, dARIMA_runtime_PKF_lags], axis=1) 
ARIMA_runtime_dailydf

,daily_PKF_allcols_ARIMA_runtime,daily_PKF_lags_ARIMA_runtime
0,0.013519,0.016309


### Daily data: all columns, unknown future

In [70]:
X_train, X_dev, dARIMA_X_test, y_train, y_dev, dARIMA_y_test = train_dev_test_split(marag_daily.drop(['TotalEntries'], axis=1), 
                                                                      marag_daily['TotalEntries'])
dARIMA_X_train = pd.concat([X_train, X_dev])
dARIMA_X_train.sort_index(inplace=True)
dARIMA_y_train = pd.concat([y_train, y_dev])
dARIMA_y_train.sort_index(inplace=True)

In [72]:
dARIMA_PUF_model = daily_ARIMA_model.fit(dARIMA_y_train, exogenous=dARIMA_X_train)

In [78]:
start_ARIMA_dPUF_allcols = time.time()
dARIMA_UF_allcols_df, dARIMA_UF_allcols_confint = ARIMA_future_preds(daily_nolags_marag, dARIMA_X_train, dARIMA_X_test, dARIMA_PUF_model, exogenous=True, daily=True)

end_ARIMA_dPUF_allcols = time.time()
dARIMA_runtime_PUF_allcols = end_ARIMA_dPUF_allcols - start_ARIMA_dPUF_allcols
dARIMA_runtime_PUF_allcols = pd.DataFrame(pd.Series(dARIMA_runtime_PUF_allcols, name = 'daily_PUF_allcols_ARIMA_runtime'))
dARIMA_UF_allcols_df.columns = ['DayEntr_PUF_allcols']
dARIMA_UF_allcols_confint.columns = ['daily_PUF_allcols_min','daily_PUF_allcols_max']


 79%|███████▉  | 122/154 [00:03<00:00, 32.27it/s]/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/numpy/linalg/linalg.py:1855: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)

 82%|████████▏ | 126/154 [00:03<00:00, 32.76it/s]/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/numpy/linalg/linalg.py:1855: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)
/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/numpy/linalg/linalg.py:1855: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)
/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/numpy/linalg/linalg.py:1855: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)
/home/quark/anaconda3/envs/mpro/lib/python3.6/site-packages/numpy/linalg/linalg.py:1855: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S >

In [80]:
ARIMA_dailyresults_df = ARIMA_dailyresults_df.merge(dARIMA_UF_allcols_df, how='left', on='Timestamp')
ARIMA_confint_dailydf = ARIMA_confint_dailydf.merge(dARIMA_UF_allcols_confint, how='left', on='Timestamp')
ARIMA_dailyresults_df.head()

,Observed_values,DayEntr_PKF_allcols,DayEntr_PKF_lags,DayEntr_PUF_allcols
Timestamp,,,,
2018-09-24,5719,9265.802845,9516.854425,9.265803e+03
2018-09-25,9998,8657.545503,9135.138861,1.508744e+07
2018-09-26,9816,9453.451855,10047.471310,-2.473273e+10
2018-09-27,10087,9591.111601,10130.052655,-1.648085e+12
2018-09-28,10742,10171.307107,10093.401038,-6.181053e+13


In [81]:
ARIMA_confint_dailydf.head()

,daily_PKF_allcols_min,daily_PKF_allcols_max,daily_PKF_lags_min,daily_PKF_lags_max,Daily_PUF_allcols_min,Daily_PUF_allcols_max
Timestamp,,,,,,
2018-09-24,7222.918945,11308.686745,7008.795707,12024.913143,7.222919e+03,1.130869e+04
2018-09-25,6609.711955,10705.379051,6558.218591,11712.059131,1.508539e+07,1.508949e+07
2018-09-26,7401.579718,11505.323992,7436.682562,12658.260058,-2.473273e+10,-2.473273e+10
2018-09-27,7535.942898,11646.280304,7502.434170,12757.671141,-1.648085e+12,-1.648085e+12
2018-09-28,8113.446637,12229.167576,7457.378367,12729.423710,-6.181053e+13,-6.181053e+13


In [82]:
ARIMA_runtime_dailydf = pd.concat([ARIMA_runtime_dailydf, dARIMA_runtime_PUF_allcols], axis=1) 
ARIMA_runtime_dailydf

,daily_PKF_allcols_ARIMA_runtime,daily_PKF_lags_ARIMA_runtime,daily_PUF_allcols_ARIMA_runtime
0,0.013519,0.016309,4.349259


### Daily data: just lags, unknown future

In [86]:
lags_dPUF_df = marag_daily.drop(['T', 'PPT', 'Weekday/Weekend'], axis = 1)

X_train, X_dev, ARIMA_dlags_X_test, y_train, y_dev, ARIMA_dlags_y_test = train_dev_test_split(lags_dPUF_df.drop(['TotalEntries'], axis=1), 
                                                                      lags_dPUF_df['TotalEntries'])

ARIMA_dlags_X_train = pd.concat([X_train, X_dev])
ARIMA_dlags_X_train.sort_index(inplace=True)
ARIMA_dlags_y_train = pd.concat([y_train, y_dev])
ARIMA_dlags_y_train.sort_index(inplace=True)

In [87]:
dARIMA_PUF_lags_model = daily_ARIMA_model.fit(ARIMA_dlags_y_train, exogenous=ARIMA_dlags_X_train)

In [94]:
start_dARIMA_PUF_lags = time.time()
dARIMA_lags_y_hat, dARIMA_lags_confint_y  = ARIMA_future_preds(daily_nolags_marag, ARIMA_dlags_X_train, ARIMA_dlags_X_test, dARIMA_PUF_model, exogenous=True, daily=True, justlags=True)

end_dARIMA_PUF_lags = time.time()
dARIMA_runtime_PUF_lags = end_dARIMA_PUF_lags - start_dARIMA_PUF_lags
dARIMA_runtime_PUF_lags = pd.DataFrame(pd.Series(dARIMA_runtime_PUF_lags, name = 'daily_PUF_lags_ARIMA_runtime'))
dARIMA_lags_y_hat.columns = ['DayEntr_PUF_lags']
dARIMA_lags_confint_y.columns = ['daily_PUF_lags_min','daily_PUF_lags_max']



  0%|          | 0/154 [00:00<?, ?it/s]

  5%|▍         | 7/154 [00:00<00:02, 58.87it/s]

  8%|▊         | 13/154 [00:00<00:02, 57.61it/s]

 12%|█▏        | 18/154 [00:00<00:02, 52.71it/s]

 16%|█▌        | 24/154 [00:00<00:02, 53.08it/s]

 19%|█▉        | 29/154 [00:00<00:02, 52.11it/s]

 23%|██▎       | 35/154 [00:00<00:02, 52.38it/s]

 26%|██▌       | 40/154 [00:00<00:02, 50.86it/s]

 30%|██▉       | 46/154 [00:00<00:02, 51.29it/s]

 34%|███▍      | 52/154 [00:01<00:01, 51.56it/s]

 38%|███▊      | 58/154 [00:01<00:01, 51.98it/s]

 42%|████▏     | 64/154 [00:01<00:01, 51.76it/s]

 45%|████▌     | 70/154 [00:01<00:01, 51.55it/s]

 49%|████▉     | 76/154 [00:01<00:01, 48.32it/s]

 53%|█████▎    | 81/154 [00:01<00:01, 46.82it/s]

 56%|█████▌    | 86/154 [00:01<00:01, 47.53it/s]

 59%|█████▉    | 91/154 [00:01<00:01, 47.82it/s]

 62%|██████▏   | 96/154 [00:01<00:01, 47.48it/s]

 66%|██████▌   | 101/154 [00:02<00:01, 44.30it/s]

 69%|██████▉   | 106/154 [00:02<00:01, 45.43it/s]

 72%|█

In [95]:
ARIMA_dailyresults_df = ARIMA_dailyresults_df.merge(dARIMA_lags_y_hat, how='left', on='Timestamp')
ARIMA_confint_dailydf = ARIMA_confint_dailydf.merge(dARIMA_lags_confint_y, how='left', on='Timestamp')
ARIMA_dailyresults_df.head()

,Observed_values,DayEntr_PKF_allcols,DayEntr_PKF_lags,DayEntr_PUF_allcols,DayEntr_PUF_lags
Timestamp,,,,,
2018-09-24,5719,9265.802845,9516.854425,9.265803e+03,9516.854425
2018-09-25,9998,8657.545503,9135.138861,1.508744e+07,9784.049681
2018-09-26,9816,9453.451855,10047.471310,-2.473273e+10,9794.004218
2018-09-27,10087,9591.111601,10130.052655,-1.648085e+12,10360.092022
2018-09-28,10742,10171.307107,10093.401038,-6.181053e+13,10238.413244


In [107]:
ARIMA_runtime_dailydf = pd.concat([ARIMA_runtime_dailydf, dARIMA_runtime_PUF_lags], axis=1) 
ARIMA_runtime_dailydf

,daily_PKF_allcols_ARIMA_runtime,daily_PKF_lags_ARIMA_runtime,daily_PUF_allcols_ARIMA_runtime,daily_PUF_lags_ARIMA_runtime
0,0.013519,0.016309,4.349259,3.26171


### Daily frequency: not using exogenous data

In [98]:
X_train, X_dev, dARIMA_X_test, y_train, y_dev, dARIMA_y_test = train_dev_test_split(marag_daily.drop(['TotalEntries'], axis=1), 
                                                                      marag_daily['TotalEntries'])
dARIMA_X_train = pd.concat([X_train, X_dev])
dARIMA_X_train.sort_index(inplace=True)
dARIMA_y_train = pd.concat([y_train, y_dev])
dARIMA_y_train.sort_index(inplace=True)

In [99]:
start_noexo_daily = time.time()

dARIMA_noexo_df, dARIMA_noexo_confint = ARIMA_future_preds(daily_nolags_marag, dARIMA_X_train, dARIMA_X_test, noexo_dARIMA_model)

end_noexo_daily = time.time()
dARIMA_runtime_noexo = end_noexo_daily - start_noexo_daily
dARIMA_runtime_noexo = pd.DataFrame(pd.Series(dARIMA_runtime_noexo, name='noexo_dARIMA_runtime'))
dARIMA_noexo_df.columns = ['DayEntr_noexo']
dARIMA_noexo_confint.columns = ['daily_noexo_min', 'daily_noexo_max']

In [100]:
ARIMA_dailyresults_df = ARIMA_dailyresults_df.merge(dARIMA_noexo_df, how='left', on='Timestamp')
ARIMA_confint_dailydf = ARIMA_confint_dailydf.merge(dARIMA_noexo_confint, how='left', on='Timestamp')
ARIMA_dailyresults_df.head()

,Observed_values,DayEntr_PKF_allcols,DayEntr_PKF_lags,DayEntr_PUF_allcols,DayEntr_PUF_lags,DayEntr_noexo
Timestamp,,,,,,
2018-09-24,5719,9265.802845,9516.854425,9.265803e+03,9516.854425,8148.180530
2018-09-25,9998,8657.545503,9135.138861,1.508744e+07,9784.049681,8898.529572
2018-09-26,9816,9453.451855,10047.471310,-2.473273e+10,9794.004218,8567.972420
2018-09-27,10087,9591.111601,10130.052655,-1.648085e+12,10360.092022,8194.394256
2018-09-28,10742,10171.307107,10093.401038,-6.181053e+13,10238.413244,8164.736295


In [108]:
ARIMA_runtime_dailydf = pd.concat([ARIMA_runtime_dailydf, dARIMA_runtime_noexo], axis=1) 
ARIMA_runtime_dailydf

,daily_PKF_allcols_ARIMA_runtime,daily_PKF_lags_ARIMA_runtime,daily_PUF_allcols_ARIMA_runtime,daily_PUF_lags_ARIMA_runtime,noexo_dARIMA_runtime
0,0.013519,0.016309,4.349259,3.26171,0.025593


In [109]:
ARIMA_dailyresults_df.to_csv('./../../../Databases/results_data/ARIMA_dailyresults.csv')
ARIMA_confint_dailydf.to_csv('./../../../Databases/results_data/ARIMA_dailyconfint.csv')
ARIMA_runtime_dailydf.to_csv('./../../../Databases/results_data/ARIMA_runtime_dailyscenarios.csv', index=False)